In [1]:
import os
import requests
from shapely.geometry import shape, Polygon, MultiPolygon
from shapely.ops import unary_union
import geopandas as gpd
import pandas as pd
from datetime import datetime
import folium

# Set the environment variable to use Shapely instead of PyGEOS
os.environ['USE_PYGEOS'] = '0'

# Define constants
API_URL = 'https://mesonet.agron.iastate.edu/api/1/nws/spc_outlook.geojson'
DATE = datetime(2018, 5, 4)
CYCLE = 16
THRESHOLD_LEVELS = ['MRGL', 'SLGT', 'ENH', 'MDT', 'HIGH']

# Load NYS boundary shapefile and reproject if necessary
nys_boundary_path = '/nfs/home11/ugrad/2020/tr588861/SWRCC/State_Shapefiles/state/State.shp'
zip_population_path = '/nfs/home11/ugrad/2020/tr588861/SWRCC/State_Shapefiles/zip_codes/zip_population.shp'

nys_boundary = gpd.read_file(nys_boundary_path)
nys_boundary.to_crs(epsg=4326, inplace=True)  # Reproject to WGS84 if needed
nys_boundary_union = unary_union(nys_boundary.geometry)

zip_population = gpd.read_file(zip_population_path)

# Calculate the total area and population of New York State
total_area = nys_boundary_union.area * (111 ** 2)
total_population = zip_population['Population'].sum()

# Function to fetch and process data for the given date
def fetch_and_process_data(date):
    params = {
        'day': 1,
        'valid': date.strftime('%Y-%m-%dT00:00'),
        'cycle': CYCLE,
        'outlook_type': 'C'
    }
    
    response = requests.get(API_URL, params=params)
    data = response.json()
    
    # Dictionary to track the highest threshold level for each zip code
    zip_highest_threshold = {}
    
    # Dictionary to store the area sum under each threshold level
    threshold_areas = {level: 0 for level in THRESHOLD_LEVELS}
    threshold_polygons = {level: [] for level in THRESHOLD_LEVELS}

    for feature in data['features']:
        properties = feature['properties']
        geometry = feature['geometry']
        
        if properties['category'] == 'CATEGORICAL':
            threshold = properties['threshold']
            
            if threshold not in THRESHOLD_LEVELS:
                continue

            # Convert the geometry to a Shapely object (handling MultiPolygon)
            polygons = shape(geometry)
            if isinstance(polygons, Polygon):
                polygons = [polygons]
            elif isinstance(polygons, MultiPolygon):
                polygons = list(polygons.geoms)

            for polygon in polygons:
                if polygon.is_valid and polygon.intersects(nys_boundary_union):
                    # Calculate the intersection with NYS boundary
                    intersection = polygon.intersection(nys_boundary_union)
                    
                    if intersection.is_empty:
                        continue
                    
                    # Add intersection polygon to the list for the current threshold
                    threshold_polygons[threshold].append(intersection)
                    
                    # Check which zip codes intersect with the intersection polygon
                    for idx, zip_geom in zip_population.iterrows():
                        if zip_geom.geometry.intersects(intersection):
                            zip_code = zip_geom['ZCTA5CE10']  # Assuming the column name for zip code
                            current_level_index = THRESHOLD_LEVELS.index(threshold)
                            
                            # Update the highest threshold level for this zip code
                            if zip_code in zip_highest_threshold:
                                previous_level_index = THRESHOLD_LEVELS.index(zip_highest_threshold[zip_code])
                                if current_level_index < previous_level_index:
                                    zip_highest_threshold[zip_code] = threshold
                            else:
                                zip_highest_threshold[zip_code] = threshold

    # Combine polygons for each threshold level and calculate areas
    for level in THRESHOLD_LEVELS:
        if threshold_polygons[level]:
            combined_polygons = unary_union(threshold_polygons[level])
            threshold_areas[level] = combined_polygons.area * (111 ** 2)
    
    # Initialize a dictionary to store the population sum under each threshold level
    population_sum = {level: 0 for level in THRESHOLD_LEVELS}

    # Sum the population based on the highest threshold level assigned
    for zip_code, highest_level in zip_highest_threshold.items():
        highest_index = THRESHOLD_LEVELS.index(highest_level)
        population = zip_population.loc[zip_population['ZCTA5CE10'] == zip_code, 'Population'].sum()
        for level in THRESHOLD_LEVELS[:highest_index + 1]:
            population_sum[level] += population
    
    # Aggregate area to include higher levels
    for i in range(len(THRESHOLD_LEVELS) - 1, 0, -1):
        threshold_areas[THRESHOLD_LEVELS[i - 1]] += threshold_areas[THRESHOLD_LEVELS[i]]

    return threshold_areas, population_sum, data

# Fetch and process data
threshold_areas, population_data, data = fetch_and_process_data(DATE)

# Calculate percentages
area_percentages = {level: (threshold_areas[level] / total_area) * 100 for level in THRESHOLD_LEVELS}
population_percentages = {level: (population_data[level] / total_population) * 100 for level in THRESHOLD_LEVELS}

# Combine area, population, and percentage data into a single table
output_data = {
    'Risk Level': THRESHOLD_LEVELS,
    'Area (Sq. Miles)': [threshold_areas[level] for level in THRESHOLD_LEVELS],
    'Area Percentage (%)': [area_percentages[level] for level in THRESHOLD_LEVELS],
    'Population': [population_data[level] for level in THRESHOLD_LEVELS],
    'Population Percentage (%)': [population_percentages[level] for level in THRESHOLD_LEVELS]
}

output_df = pd.DataFrame(output_data)

# Print the combined table
print(output_df)

# Create a folium map to visualize the data
m = folium.Map(location=[43.0, -75.0], zoom_start=7)

# Add NYS boundary to the map
folium.GeoJson(
    nys_boundary,
    style_function=lambda x: {'color': 'black', 'weight': 2}
).add_to(m)

# Add polygons to the map with colors based on the risk level
for feature in data['features']:
    properties = feature['properties']
    geometry = shape(feature['geometry'])
    
    if properties['category'] == 'CATEGORICAL' and properties['threshold'] in THRESHOLD_LEVELS:
        color = {
            'TSTM': '#c0e8c0',
            'MRGL': '#7fc57f',
            'SLGT': '#f6f67f',
            'ENH': '#e6c27f',
            'MDT': '#e67f7f',
            'HIGH': '#ff7fff'
        }[properties['threshold']]
        
        folium.GeoJson(
            geometry,
            style_function=lambda x, color=color: {
                'color': color,
                'fillOpacity': 0.5
            }
        ).add_to(m)

# Save the map to an HTML file
m.save('nys_risk_levels_map.html')

/tmp/ipykernel_3950026/1447396954.py:5: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, this switch should be smooth. However, if you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


  Risk Level  Area (Sq. Miles)  Area Percentage (%)  Population  \
0       MRGL     180957.317326            94.235548  11110198.0   
1       SLGT     179123.252142            93.280438   7224955.0   
2        ENH     154432.451577            80.422428   5535698.0   
3        MDT          0.000000             0.000000         0.0   
4       HIGH          0.000000             0.000000         0.0   

   Population Percentage (%)  
0                  55.740743  
1                  36.248171  
2                  27.773035  
3                   0.000000  
4                   0.000000  


In [2]:
zip_population.head()

,fid,ZCTA5CE10,GEOID10,CLASSFP10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,Code,Year,Population,geometry
0,1.0,14135,14135,B5,G6350,S,2676215.0,0.0,+42.4876943,-079.2408999,14135,2022.0,141.0,"POLYGON ((-79.25321 42.48592, -79.25315 42.485..."
1,2.0,14136,14136,B5,G6350,S,59969868.0,10885759.0,+42.5173793,-079.1748123,14136,2022.0,5122.0,"POLYGON ((-79.24982 42.53745, -79.17963 42.560..."
2,3.0,14138,14138,B5,G6350,S,121375614.0,1211367.0,+42.3795121,-079.0366555,14138,2022.0,1864.0,"POLYGON ((-79.15951 42.37216, -79.15920 42.375..."
3,4.0,14139,14139,B5,G6350,S,61792609.0,82395.0,+42.7184053,-078.5414018,14139,2022.0,2074.0,"POLYGON ((-78.61243 42.70249, -78.60951 42.703..."
4,5.0,14141,14141,B5,G6350,S,182617300.0,392658.0,+42.5249667,-078.7120755,14141,2022.0,7492.0,"POLYGON ((-78.80503 42.51980, -78.80101 42.519..."
